In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_funding_settlements_for_day(date_str="2025-05-30"):
    """Získa funding settlement časy pre daný deň"""
    settlements = []
    base_date = datetime.strptime(date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc)
    
    # Funding časy: 04:00, 12:00, 20:00 UTC
    for hour in [4, 12, 20]:
        settlement_time = base_date.replace(hour=hour, minute=0, second=0, microsecond=0)
        settlements.append(settlement_time)
    
    return settlements

In [3]:
def calculate_predicted_funding_at_time(symbol, target_time, settlement_start):
    """Vypočíta predicted funding rate v konkrétnom čase"""
    
    # Časové okno od settlement do target_time
    start_time = settlement_start.strftime("%Y-%m-%dT%H:%M:%S.000Z")
    end_time = target_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
    
    try:
        # Počet minút v okne + buffer
        time_diff = target_time - settlement_start
        minutes_needed = int(time_diff.total_seconds() / 60) + 10
        count = min(minutes_needed, 500)
        
        # Získaj Premium Index dáta
        url_pi = f"https://www.bitmex.com/api/v1/trade?symbol=.{symbol}PI&count={count}&reverse=true&endTime={end_time}"
        resp_pi = requests.get(url_pi)
        
        if resp_pi.status_code != 200:
            return None
            
        pi_data = resp_pi.json()
        if not pi_data:
            return None
        
        # Spracuj dáta
        df_pi = pd.DataFrame(pi_data)
        df_pi['timestamp'] = pd.to_datetime(df_pi['timestamp'])
        
        # Filtruj dáta od settlement_start do target_time
        df_pi_filtered = df_pi[
            (df_pi['timestamp'] >= settlement_start) & 
            (df_pi['timestamp'] <= target_time)
        ]
        
        if df_pi_filtered.empty:
            return None
        
        # Vypočítaj priemer Premium Index
        avg_premium_index = df_pi_filtered['price'].mean()
        data_points = len(df_pi_filtered)
        
        # Aplikuj BitMEX vzorec
        interest_rate = 0.0001  # 0.01%
        diff = interest_rate - avg_premium_index
        clamped_value = max(-0.0005, min(0.0005, diff))
        predicted_funding = avg_premium_index + clamped_value
        
        return {
            'timestamp': target_time,
            'predicted_funding': predicted_funding,
            'predicted_funding_percent': predicted_funding * 100,
            'premium_index': avg_premium_index,
            'data_points': data_points,
            'hours_since_settlement': time_diff.total_seconds() / 3600
        }
        
    except Exception as e:
        print(f"Chyba pre čas {target_time}: {e}")
        return None